In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import warnings
import gc

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from xgboost import XGBClassifier

warnings.filterwarnings('ignore')

In [ ]:
def evaluate_model(model, x, y):
    y_pred = model.predict(x)
    acc = accuracy_score(y, y_pred)
    return {'accuracy' : acc}

In [ ]:
seed = 47

In [ ]:
train_df = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2022/train.csv')

In [ ]:
train_df.drop(['row_id'], axis=1, inplace=True)
target = train_df.pop('target')

In [ ]:
le = LabelEncoder()
target = le.fit_transform(target)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(train_df, target, test_size=0.2, random_state=seed)

# XGBoost Baseline

In [ ]:
def get_xgb_baseline(params={}):
    return XGBClassifier(**params,
                         random_state=seed,
                         tree_method='gpu_hist',
                         predictor='gpu_predictor',
                         eval_metric='auc',
                         objective='multi:softprob')

In [ ]:
model = get_xgb_baseline()
model.fit(x_train, y_train)
results = evaluate_model(model, x_test, y_test)
print(results)

# Feature Engineering

Here wee will experiment creating synthetic features using central tendency statistics.


In [ ]:
geomean = lambda x, axis : np.exp(np.mean(np.log(x), axis=axis))
harmonic_mean = lambda x, axis : len(x) / np.sum(1.0/x, axis=axis) 

funcs = {'mean' : np.mean, 
         'std' : np.std, 
         'var' : np.var, 
         'geo_mean' : geomean, 
         'harmonic_mean' : harmonic_mean, 
         'median' : np.median,
         'none': None}

In [ ]:
results, names = list(), list()

for key in funcs.keys():
    x_train = train_df.copy()
    if funcs[key] is not None:
        x_train[key] = funcs[key](x_train, axis=1)
    x_train, x_test, y_train, y_test = train_test_split(x_train, target, test_size=0.2, random_state=seed)       
    model = get_xgb_baseline()

    model.fit(x_train, y_train)
    result = evaluate_model(model, x_test, y_test)
    names.append(key)
    results.append(result['accuracy'])
    
for name, score in zip(names, results):
    print('>%s: %f' % (name, score))

index_best = np.argmax(score)
print('Best result is of:', names[index_best], 'with score:', results[index_best])

# Individual parameter Search

In [ ]:
x_train = train_df.copy()
x_train['mean'] = np.mean(x_train, axis=1)
x_train, x_test, y_train, y_test = train_test_split(x_train, target, test_size=0.2, random_state=seed)

In [ ]:
params = {}

<h2>1 - Testing different number of estimators</h2>


In [ ]:
results_trees = {}
trees = [50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 600, 700, 800, 900, 950, 975, 1000, 1025, 1050, 1100, 1150, 1290, 1295, 1300, 1305, 1310, 1315, 1325, 2000, 3000, 4000, 5000, 10000]

for n in trees:
    params['n_estimators'] = n
    model = get_xgb_baseline(params)
    model.fit(x_train, y_train)
    result = evaluate_model(model, x_test, y_test)
    results_trees[n] = result['accuracy']
    print('n_estimators:', n, 'accuracy:', results_trees[n])

best_nestimator = max(results_trees, key=results_trees.get)
print('\nBest n_estimators:', best_nestimator, 'Accuracy score:', results_trees[best_nestimator])

<h2>2 - Testing different max_depth</h2>

In [ ]:
results_max_depths = {}
params['n_estimators'] = best_nestimator
max_depths = [i for i in range(1,10)]

for max_depth in max_depths:
    params['max_depth'] = max_depth
    model = get_xgb_baseline(params)
    model.fit(x_train, y_train)
    result = evaluate_model(model, x_test, y_test)
    results_max_depths[max_depth] = result['accuracy']
    print('max_depth:', max_depth, 'accuracy:', results_max_depths[max_depth])

best_max_depth = max(results_max_depths, key=results_max_depths.get)
print('\nBest max_depth:', best_max_depth, 'Accuracy score:', results_max_depths[best_max_depth])

<h2>3 - Testing different subsamples</h2>


In [ ]:
results_subsamples = {}
params['max_depth'] = best_max_depth
subsamples = [i for i in np.arange(0.1, 1.1, 0.1)]

for subsample in subsamples:
    params['subsample'] = subsample
    model = model = get_xgb_baseline(params)
    model.fit(x_train, y_train)
    result = evaluate_model(model, x_test, y_test)
    results_subsamples[subsample] = result['accuracy']
    print('subsample:', subsample, 'accuracy:', results_subsamples[subsample])

best_subsample = max(results_subsamples, key=results_subsamples.get)
print('\nBest subsample:', best_subsample, 'Accuracy score:', results_subsamples[best_subsample])

<h2>4 - Testing different learning rates</h2>

In [ ]:
results_etas = {}
params['subsample'] = best_subsample
etas = [0.0001, 0.001, 0.003, 0.005, 0.01, 0.03, 0.05, 0.1, 0.12, 0.13, 0.3, 0.5, 1.0]

for eta in etas:
    params['eta'] = eta
    model = get_xgb_baseline(params)
    model.fit(x_train, y_train)
    result = evaluate_model(model, x_test, y_test)
    results_etas[eta] = result['accuracy']
    print('eta:', eta, 'accuracy:', results_etas[eta])

best_eta = max(results_etas, key=results_etas.get)
print('\nBest eta:', best_eta, 'Accuracy score:', results_etas[best_eta])

<h2>5 - Testing different number of features</h2>

In [ ]:
results_colsample_bytrees = {}
params['eta'] = best_eta
colsample_bytrees = [i for i in np.arange(0.1, 1.1, 0.1)]

for colsample_bytree in colsample_bytrees:
    params['colsample_bytree'] = colsample_bytree
    model = get_xgb_baseline(params)
    model.fit(x_train, y_train)
    result = evaluate_model(model, x_test, y_test)
    results_colsample_bytrees[colsample_bytree] = result['accuracy']
    print('colsample_bytree:', colsample_bytree, ':', results_colsample_bytrees[colsample_bytree])

best_colsample_bytree = max(results_colsample_bytrees, key=results_colsample_bytrees.get)
print('\nBest colsample_bytree:', best_colsample_bytree, 'Accuracy score:', results_colsample_bytrees[best_colsample_bytree])

<h2>6 - Testing different values for min_child_weight</h2>

In [ ]:
results_min_child_weight = {}
params['colsample_bytree'] = best_colsample_bytree
min_child_weights = [i for i in range(1,10)]

for min_child_weight in min_child_weights:
    params['min_child_weight'] = min_child_weight
    model = get_xgb_baseline(params)
    model.fit(x_train, y_train)
    result = evaluate_model(model, x_test, y_test)
    results_min_child_weight[min_child_weight] = result['accuracy']
    print('min_child_weight:', min_child_weight, 'accuracy:', results_min_child_weight[min_child_weight])

best_min_child_weight = max(results_min_child_weight, key=results_min_child_weight.get)
print('\nBest min_child_weight:', best_min_child_weight, 'Accuracy score:', results_min_child_weight[best_min_child_weight])

<h2>7 - Testing different values for gamma</h2>

In [ ]:
results_gamma = {}
params['min_child_weight'] = best_min_child_weight
gammas = [0.01, 0.02, 0.03, 0.1, 0.3, 0.5, 1, 1.1, 1.5, 2, 5, 7, 9, 10]

for gamma in gammas:
    params['gamma'] = gamma
    model = get_xgb_baseline(params)
    model.fit(x_train, y_train)
    result = evaluate_model(model, x_test, y_test)
    results_gamma[gamma] = result['accuracy']
    print('gamma:', gamma, 'accuracy:', results_gamma[gamma])

best_gamma = max(results_gamma, key=results_gamma.get)
print('\nBest gamma:', best_gamma, 'Accuracy score:', results_gamma[best_gamma])

In [ ]:
params['gamma'] = best_gamma
print('Best Hyperparameters:', params)

In [ ]:
params.pop('gamma')

# Submission

In [ ]:
model = get_xgb_baseline(params)
model.fit(x_train, y_train)
result = evaluate_model(model, x_test, y_test)
print(result)

In [ ]:
del x_train, x_test, y_train, y_test, train_df
gc.collect()

In [ ]:
test_df = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2022/test.csv')

In [ ]:
ids = test_df.pop('row_id').values
test_df['mean'] = np.mean(test_df, axis=1)
target = le.inverse_transform(model.predict(test_df))
submission = pd.DataFrame({'row_id' : ids, 'target' : target})

In [ ]:
submission.head()

In [ ]:
submission.to_csv('submission.csv', index=False)